# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

## 필요없는 features 제거

In [4]:
# 필요없는 features 제거
data.drop(['PassengerId', 'Cabin', 'Cabin2', 'Age', 'Name', 'Combi'], axis=1, inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   HomePlanet    8693 non-null   object
 1   CryoSleep     8693 non-null   bool  
 2   Cabin1        8590 non-null   object
 3   Cabin3        8590 non-null   object
 4   Destination   8693 non-null   object
 5   VIP           8693 non-null   bool  
 6   RoomService   8693 non-null   int64 
 7   FoodCourt     8693 non-null   int64 
 8   ShoppingMall  8693 non-null   int64 
 9   Spa           8693 non-null   int64 
 10  VRDeck        8693 non-null   int64 
 11  Transported   8693 non-null   bool  
dtypes: bool(3), int64(5), object(4)
memory usage: 636.8+ KB


## 처리하기 힘든 결측값 제거

In [6]:
data.isna().sum()

HomePlanet        0
CryoSleep         0
Cabin1          103
Cabin3          103
Destination       0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
dtype: int64

In [7]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## Boolean 캐스팅

In [8]:
# Cabin3의 값을 변환
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
data['Cabin3'] = data['Cabin3'].astype(bool)

## 원핫인코딩

In [9]:
# 원핫인코딩
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CryoSleep      8590 non-null   bool 
 1   Cabin3         8590 non-null   bool 
 2   VIP            8590 non-null   bool 
 3   RoomService    8590 non-null   int64
 4   FoodCourt      8590 non-null   int64
 5   ShoppingMall   8590 non-null   int64
 6   Spa            8590 non-null   int64
 7   VRDeck         8590 non-null   int64
 8   Transported    8590 non-null   bool 
 9   Earth          8590 non-null   uint8
 10  Europa         8590 non-null   uint8
 11  Mars           8590 non-null   uint8
 12  55 Cancri e    8590 non-null   uint8
 13  PSO J318.5-22  8590 non-null   uint8
 14  TRAPPIST-1e    8590 non-null   uint8
 15  A              8590 non-null   uint8
 16  B              8590 non-null   uint8
 17  C              8590 non-null   uint8
 18  D              8590 non-null   uint8
 19  E     

## 스케일링

In [13]:
# 스케일링
col = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [14]:
data_scaled(data, col)

,CryoSleep,Cabin3,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Earth,...,PSO J318.5-22,TRAPPIST-1e,A,B,C,D,E,F,G,T
0,False,True,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,0,...,0,1,0,1,0,0,0,0,0,0
1,False,False,False,-0.168530,-0.275148,-0.241196,0.220460,-0.224520,True,1,...,0,1,0,0,0,0,0,1,0,0
2,False,False,True,-0.268567,1.959032,-0.282832,5.745469,-0.220106,False,0,...,0,1,1,0,0,0,0,0,0,0
3,False,False,False,-0.333743,0.522818,0.335048,2.711463,-0.092988,False,0,...,0,1,1,0,0,0,0,0,0,0
4,False,False,False,0.125518,-0.236941,-0.031350,0.234796,-0.261596,True,1,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,True,True,-0.333743,3.990274,-0.282832,1.200732,-0.198037,False,0,...,0,0,1,0,0,0,0,0,0,0
8689,True,False,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,False,1,...,1,0,0,0,0,0,0,0,1,0
8690,False,False,False,-0.333743,-0.280785,2.834877,-0.270573,-0.263361,True,1,...,0,1,0,0,0,0,0,0,1,0
8691,False,False,False,-0.333743,0.376253,-0.282832,0.044835,2.592370,False,0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
data.columns

Index(['CryoSleep', 'Cabin3', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Earth', 'Europa',
       'Mars', '55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'T'],
      dtype='object')

# 데이터셋 분리

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(['Transported'], axis=1),
                                                    data.Transported, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

# XGBoost

In [17]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [18]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [19]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.89623	validation_1-auc:0.86179                                                                  
[1]	validation_0-auc:0.90991	validation_1-auc:0.87339                                                                  
[2]	validation_0-auc:0.91703	validation_1-auc:0.87605                                                                  
[3]	validation_0-auc:0.92006	validation_1-auc:0.87942                                                                  
[4]	validation_0-auc:0.92569	validation_1-auc:0.88109                                                                  
[5]	validation_0-auc:0.92683	validation_1-auc:0.87976                                                                  
[6]	validation_0-auc:0.92925	validation_1-auc:0.88138                                                                  
[7]	validation_0-auc:0.93011	validation_1-auc:0.88092                                                                  
[8]	validation_0-auc:0.93184	validation_

[1]	validation_0-auc:0.91172	validation_1-auc:0.86542                                                                  
[2]	validation_0-auc:0.91695	validation_1-auc:0.87015                                                                  
[3]	validation_0-auc:0.91813	validation_1-auc:0.86942                                                                  
[4]	validation_0-auc:0.92279	validation_1-auc:0.87045                                                                  
[5]	validation_0-auc:0.92554	validation_1-auc:0.87216                                                                  
[6]	validation_0-auc:0.92747	validation_1-auc:0.87427                                                                  
[7]	validation_0-auc:0.92946	validation_1-auc:0.87728                                                                  
[8]	validation_0-auc:0.93059	validation_1-auc:0.87853                                                                  
[9]	validation_0-auc:0.93268	validation_

[20]	validation_0-auc:0.93775	validation_1-auc:0.89967                                                                 
[21]	validation_0-auc:0.93775	validation_1-auc:0.89967                                                                 
[22]	validation_0-auc:0.93775	validation_1-auc:0.89967                                                                 
[23]	validation_0-auc:0.93844	validation_1-auc:0.89908                                                                 
[24]	validation_0-auc:0.93844	validation_1-auc:0.89908                                                                 
[25]	validation_0-auc:0.93844	validation_1-auc:0.89908                                                                 
[26]	validation_0-auc:0.93886	validation_1-auc:0.89935                                                                 
[27]	validation_0-auc:0.93886	validation_1-auc:0.89935                                                                 
[28]	validation_0-auc:0.93886	validation

[47]	validation_0-auc:0.92002	validation_1-auc:0.88670                                                                 
[48]	validation_0-auc:0.92010	validation_1-auc:0.88693                                                                 
[49]	validation_0-auc:0.92067	validation_1-auc:0.88697                                                                 
[50]	validation_0-auc:0.92077	validation_1-auc:0.88681                                                                 
[51]	validation_0-auc:0.92089	validation_1-auc:0.88696                                                                 
[52]	validation_0-auc:0.92146	validation_1-auc:0.88728                                                                 
[53]	validation_0-auc:0.92153	validation_1-auc:0.88735                                                                 
[54]	validation_0-auc:0.92192	validation_1-auc:0.88730                                                                 
[55]	validation_0-auc:0.92218	validation

[15]	validation_0-auc:0.90933	validation_1-auc:0.87874                                                                 
[16]	validation_0-auc:0.90981	validation_1-auc:0.87943                                                                 
[17]	validation_0-auc:0.91127	validation_1-auc:0.88004                                                                 
[18]	validation_0-auc:0.91197	validation_1-auc:0.88046                                                                 
[19]	validation_0-auc:0.91281	validation_1-auc:0.88199                                                                 
[20]	validation_0-auc:0.91341	validation_1-auc:0.88188                                                                 
[21]	validation_0-auc:0.91408	validation_1-auc:0.88136                                                                 
[22]	validation_0-auc:0.91476	validation_1-auc:0.88227                                                                 
[23]	validation_0-auc:0.91530	validation

...

[30]	validation_0-auc:0.91446	validation_1-auc:0.88649                                                                 
[31]	validation_0-auc:0.91468	validation_1-auc:0.88659                                                                 
[32]	validation_0-auc:0.91482	validation_1-auc:0.88670                                                                 
[33]	validation_0-auc:0.91514	validation_1-auc:0.88659                                                                 
[34]	validation_0-auc:0.91577	validation_1-auc:0.88657                                                                 
[35]	validation_0-auc:0.91600	validation_1-auc:0.88637                                                                 
[36]	validation_0-auc:0.91641	validation_1-auc:0.88681                                                                 
[37]	validation_0-auc:0.91649	validation_1-auc:0.88675                                                                 
[38]	validation_0-auc:0.91649	validation

[18]	validation_0-auc:0.90587	validation_1-auc:0.89642                                                                 
[19]	validation_0-auc:0.90662	validation_1-auc:0.89592                                                                 
[20]	validation_0-auc:0.90695	validation_1-auc:0.89608                                                                 
[21]	validation_0-auc:0.90802	validation_1-auc:0.89673                                                                 
[22]	validation_0-auc:0.90820	validation_1-auc:0.89625                                                                 
[23]	validation_0-auc:0.90890	validation_1-auc:0.89656                                                                 
[24]	validation_0-auc:0.90919	validation_1-auc:0.89706                                                                 
[25]	validation_0-auc:0.90949	validation_1-auc:0.89727                                                                 
[26]	validation_0-auc:0.90950	validation

[86]	validation_0-auc:0.92229	validation_1-auc:0.90031                                                                 
[87]	validation_0-auc:0.92258	validation_1-auc:0.90039                                                                 
[88]	validation_0-auc:0.92290	validation_1-auc:0.90041                                                                 
[0]	validation_0-auc:0.89611	validation_1-auc:0.84901                                                                  
[1]	validation_0-auc:0.90455	validation_1-auc:0.86532                                                                  
[2]	validation_0-auc:0.90639	validation_1-auc:0.87198                                                                  
[3]	validation_0-auc:0.91215	validation_1-auc:0.87573                                                                  
[4]	validation_0-auc:0.91890	validation_1-auc:0.88012                                                                  
[5]	validation_0-auc:0.92096	validation_

[15]	validation_0-auc:0.93766	validation_1-auc:0.88411                                                                 
[16]	validation_0-auc:0.93788	validation_1-auc:0.88389                                                                 
[17]	validation_0-auc:0.93975	validation_1-auc:0.88495                                                                 
[18]	validation_0-auc:0.94087	validation_1-auc:0.88588                                                                 
[19]	validation_0-auc:0.94101	validation_1-auc:0.88610                                                                 
[20]	validation_0-auc:0.94135	validation_1-auc:0.88637                                                                 
[21]	validation_0-auc:0.94171	validation_1-auc:0.88605                                                                 
[22]	validation_0-auc:0.94248	validation_1-auc:0.88660                                                                 
[23]	validation_0-auc:0.94266	validation

[14]	validation_0-auc:0.93523	validation_1-auc:0.89697                                                                 
[15]	validation_0-auc:0.93637	validation_1-auc:0.89797                                                                 
[16]	validation_0-auc:0.93690	validation_1-auc:0.89808                                                                 
[17]	validation_0-auc:0.93750	validation_1-auc:0.89890                                                                 
[18]	validation_0-auc:0.93796	validation_1-auc:0.89932                                                                 
[19]	validation_0-auc:0.93905	validation_1-auc:0.89933                                                                 
[20]	validation_0-auc:0.93971	validation_1-auc:0.89958                                                                 
[21]	validation_0-auc:0.93994	validation_1-auc:0.89942                                                                 
[22]	validation_0-auc:0.94046	validation

In [23]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [20]:
xgbo = xgb.XGBClassifier(colsample_bytree=0.5600002390099754, gamma=1.6173143019744638,
                         learning_rate=0.1683119807428255,
                         max_depth=9, min_child_weight=5, random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.5600002390099754, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=1.6173143019744638, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.1683119807428255, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=9, max_leaves=0, min_child_weight=5, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [21]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [24]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[1978  422]
 [ 261 2170]]
정확도: 0.8586, 정밀도: 0.8372,     재현율: 0.8926, F1: 0.8640


In [25]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[814 253]
 [192 889]]
정확도: 0.7928, 정밀도: 0.7785,     재현율: 0.8224, F1: 0.7998


In [26]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[586 204]
 [135 686]]
정확도: 0.7896, 정밀도: 0.7708,     재현율: 0.8356, F1: 0.8019


In [ ]:
fi = pd.DataFrame(xgbo.feature_importances_, index=data.drop('Transported', axis=1).columns)

In [ ]:
fi.to_csv('fi_5.csv')

In [ ]:
# 